
# Siamese Network for Face Verification


In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from sklearn.datasets import fetch_lfw_pairs
from tensorflow.keras.preprocessing import image_dataset_from_directory
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
# ---------------------------
# 1. Build Embedding Model
# ---------------------------

In [ ]:
def build_embedding_model(embedding_dim=128):
    base_model = tf.keras.applications.ResNet50(
        include_top=False, 
        weights="imagenet", 
        pooling="avg", 
        input_shape=(224, 224, 3)
    )

     # Fine-tune last layers
    base_model.trainable = True
    for layer in base_model.layers[:100]:
        layer.trainable = False
    
    inputs = layers.Input(shape=(224,224,3))
    x = tf.keras.applications.resnet50.preprocess_input(inputs)
    x = base_model(x, training=True)

    # Deeper embedding head
    x = layers.Dense(512, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(embedding_dim)(x)
    x = layers.Lambda(lambda t: tf.math.l2_normalize(t, axis=1))(x)
    
    return models.Model(inputs, x, name="EmbeddingModel")

In [ ]:
# ---------------------------
# 2. Build Siamese Model (Distance-based)
# ---------------------------

In [ ]:
def build_siamese_model(embedding_model):
    input_a = layers.Input(shape=(224,224,3))
    input_b = layers.Input(shape=(224,224,3))

    emb_a = embedding_model(input_a)
    emb_b = embedding_model(input_b)

    # Calculate Euclidean distance
    distance = layers.Lambda(lambda embeddings: tf.sqrt(tf.reduce_sum(tf.square(embeddings[0] - embeddings[1]), axis=1, keepdims=True)))([emb_a, emb_b])

    return models.Model([input_a, input_b], distance, name="SiameseNet")

In [ ]:
# ---------------------------
# 3. Alternative: Binary Classification Siamese Model
# ---------------------------

In [ ]:
def build_siamese_model_binary(embedding_model):
    input_a = layers.Input(shape=(224,224,3))
    input_b = layers.Input(shape=(224,224,3))

    emb_a = embedding_model(input_a)
    emb_b = embedding_model(input_b)

    # Calculate similarity features
    cosine_sim = layers.Dot(axes=1, normalize=True)([emb_a, emb_b])
    cosine_sim = layers.Reshape((1,))(cosine_sim)
    abs_diff = layers.Lambda(lambda x: tf.abs(x[0] - x[1]))([emb_a, emb_b])
    
    # Combine features
    combined = layers.Concatenate()([cosine_sim, abs_diff])
    
    # Classification head
    x = layers.Dense(64, activation='relu')(combined)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(1, activation='sigmoid')(x)

    return models.Model([input_a, input_b], output, name="SiameseNet")

In [ ]:
# ---------------------------
# 4. Contrastive Loss for Distance-based Model
# ---------------------------

In [ ]:
class ContrastiveLoss(tf.keras.losses.Loss):
    def __init__(self, margin=1.0, **kwargs):
        super().__init__(**kwargs)
        self.margin = margin

    def call(self, y_true, y_pred):
        # y_true: 1 for same person, 0 for different persons
        # y_pred: euclidean distance
        y_true = tf.cast(y_true, tf.float32)
        
        # For same person (y_true=1): minimize distance
        same_loss = y_true * tf.square(y_pred)
        
        # For different persons (y_true=0): maximize distance up to margin
        diff_loss = (1 - y_true) * tf.square(tf.maximum(0.0, self.margin - y_pred))
        
        return tf.reduce_mean(same_loss + diff_loss)

In [ ]:
# ---------------------------
# 5. Data Loading and Preprocessing
# ---------------------------

In [ ]:
def preprocess(img):
    img = tf.image.resize(img, (224,224))
    img = tf.cast(img, tf.float32)
    return img

def make_dataset(lfw_pairs):
    X1, X2, y = lfw_pairs.pairs[:,0], lfw_pairs.pairs[:,1], lfw_pairs.target
    X1 = np.array([preprocess(img).numpy() for img in X1])
    X2 = np.array([preprocess(img).numpy() for img in X2])
    y  = np.array(y).astype("float32")
    return (X1, X2), y

In [ ]:
# Load data
print("Loading LFW dataset...")
lfw_pairs_train = fetch_lfw_pairs(subset='train', color=True, resize=0.5, download_if_missing=True)
lfw_pairs_test  = fetch_lfw_pairs(subset='test', color=True, resize=0.5, download_if_missing=True)

(train_X1, train_X2), train_y = make_dataset(lfw_pairs_train)
(test_X1, test_X2), test_y   = make_dataset(lfw_pairs_test)

print(f"Training pairs: {len(train_y)}")
print(f"Test pairs: {len(test_y)}")
print("Label distribution:", np.unique(train_y, return_counts=True))

In [ ]:
# ---------------------------
# 6. Build and Compile Models
# ---------------------------

In [ ]:
# Build embedding model
embedding_model = build_embedding_model()

# Option 1: Distance-based Siamese model
print("Building distance-based Siamese model...")
siamese_model = build_siamese_model(embedding_model)

# Option 2: Binary classification Siamese model (recommended)
print("Building binary classification Siamese model...")
siamese_model_binary = build_siamese_model_binary(embedding_model)

In [ ]:
# ---------------------------
# 7. Training Setup
# ---------------------------

In [ ]:
# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'best_siamese_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

In [ ]:
# ---------------------------
# 8. Training - Choose ONE of these options
# ---------------------------

In [ ]:
# OPTION 1: Distance-based model with Contrastive Loss
print("\n=== Training Distance-based Model ===")
siamese_model.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss=ContrastiveLoss(margin=1.0),
    metrics=["accuracy"]
)

history_distance = siamese_model.fit(
    [train_X1, train_X2], train_y,
    validation_data=([test_X1, test_X2], test_y),
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# OPTION 2: Binary Classification Model (RECOMMENDED)
print("\n=== Training Binary Classification Model ===")
siamese_model_binary.compile(
    optimizer=tf.keras.optimizers.Adam(0.0001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history_binary = siamese_model_binary.fit(
    [train_X1, train_X2], train_y,
    validation_data=([test_X1, test_X2], test_y),
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

In [ ]:
# ---------------------------
# 9. Plotting Training History
# ---------------------------

In [ ]:
def plot_training_history(history, title="Training History"):
    plt.figure(figsize=(15, 5))
    
    # Loss plot
    plt.subplot(1, 3, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{title} - Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # Accuracy plot
    plt.subplot(1, 3, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{title} - Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    # Learning rate plot
    plt.subplot(1, 3, 3)
    if 'learning_rate' in history.history:
        plt.plot(history.history['learning_rate'], label='Learning Rate')
        plt.title(f'{title} - Learning Rate')
        plt.xlabel('Epochs')
        plt.ylabel('Learning Rate')
        plt.legend()
        plt.grid(True)
    
    plt.tight_layout()
    plt.show()

# Plot results for both models
plot_training_history(history_distance, "Distance-based Model")
plot_training_history(history_binary, "Binary Classification Model")

In [ ]:
# ---------------------------
# 10. Model Evaluation
# ---------------------------

In [ ]:
def evaluate_model(model, test_data, model_type="distance"):
    test_loss, test_acc = model.evaluate(test_data[0], test_data[1], verbose=0)
    print(f"\n=== {model_type.upper()} MODEL RESULTS ===")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    
    # Get predictions
    predictions = model.predict(test_data[0])
    
    if model_type == "distance":
        # For distance model, threshold around 0.5
        threshold = 0.5
        pred_labels = (predictions.flatten() < threshold).astype(int)
    else:
        # For binary model, threshold at 0.5
        threshold = 0.5
        pred_labels = (predictions.flatten() > threshold).astype(int)
    
    # Calculate accuracy manually
    correct = np.sum(pred_labels == test_data[1])
    manual_acc = correct / len(test_data[1])
    print(f"Manual Accuracy: {manual_acc:.4f}")
    
    return test_loss, test_acc

# Evaluate both models
evaluate_model(siamese_model, ([test_X1, test_X2], test_y), "distance")
evaluate_model(siamese_model_binary, ([test_X1, test_X2], test_y), "binary")

In [ ]:
# ---------------------------
# 11. Inference Functions
# ---------------------------

In [ ]:
def compare_faces_distance(img_path1, img_path2, model, threshold=0.5):
    """For distance-based model"""
    def load_and_preprocess(path):
        img = Image.open(path).convert("RGB")
        img = img.resize((224,224))
        img = np.array(img).astype("float32")
        return img
    
    img1 = load_and_preprocess(img_path1)
    img2 = load_and_preprocess(img_path2)
    
    img1 = np.expand_dims(img1, axis=0)
    img2 = np.expand_dims(img2, axis=0)

    distance = model.predict([img1, img2], verbose=0)[0][0]
    print(f"Distance: {distance:.4f}")
    
    if distance < threshold:  # Lower distance = more similar
        print("✅ Same person")
        return True
    else:
        print("❌ Different persons")
        return False

def compare_faces_binary(img_path1, img_path2, model, threshold=0.5):
    """For binary classification model"""
    def load_and_preprocess(path):
        img = Image.open(path).convert("RGB")
        img = img.resize((224,224))
        img = np.array(img).astype("float32")
        return img
    
    img1 = load_and_preprocess(img_path1)
    img2 = load_and_preprocess(img_path2)
    
    img1 = np.expand_dims(img1, axis=0)
    img2 = np.expand_dims(img2, axis=0)

    similarity = model.predict([img1, img2], verbose=0)[0][0]
    print(f"Similarity: {similarity:.4f}")
    
    if similarity > threshold:
        print("✅ Same person")
        return True
    else:
        print("❌ Different persons")
        return False

In [ ]:
# ---------------------------
# 12. Save Models
# ---------------------------

In [ ]:
print("Saving models...")
siamese_model.save('siamese_distance_model.h5')
siamese_model_binary.save('siamese_binary_model.h5')
embedding_model.save('embedding_model.h5')
print("Models saved successfully!")

In [ ]:
# ---------------------------
# 13. Example Usage
# ---------------------------

In [ ]:
print("\n=== USAGE EXAMPLES ===")
print("For distance-based model:")
print("compare_faces_distance('path1.jpg', 'path2.jpg', siamese_model)")
print("\nFor binary classification model:")
print("compare_faces_binary('path1.jpg', 'path2.jpg', siamese_model_binary)")

print("\n=== TRAINING COMPLETE ===")
print("Key improvements made:")
print("1. ✅ Fixed architecture mismatch between model output and loss function")
print("2. ✅ Added proper contrastive loss implementation")
print("3. ✅ Implemented binary classification alternative")
print("4. ✅ Added comprehensive callbacks and monitoring")
print("5. ✅ Reduced learning rate for stable training")
print("6. ✅ Added proper evaluation and inference functions")

In [ ]:
print("
=== TRAINING COMPLETE ===")
print("Key improvements made:")
print("1. ✅ Fixed architecture mismatch between model output and loss function")
print("2. ✅ Added proper contrastive loss implementation")
print("3. ✅ Implemented binary classification alternative")
print("4. ✅ Added comprehensive callbacks and monitoring")
print("5. ✅ Reduced learning rate for stable training")
print("6. ✅ Added proper evaluation and inference functions")
